In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import umap
import dill
import featuretools as ft
import lightgbm as lgb
import janitor

from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, RobustScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from pyod.models.copod import COPOD

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

#### Used only for saving/loading session variables.

In [ ]:
#dill.dump_session("session.db")

In [ ]:
#dill.load_session("session.db")

In [ ]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

data_submission = pd.DataFrame({"User_ID" : data_test["User_ID"], "Product_ID" : data_test["Product_ID"]})
data_train.drop(["User_ID", "Product_ID"], axis=1, inplace=True)
data_test.drop(["User_ID", "Product_ID"], axis=1, inplace=True)
data_train.head()

In [ ]:
data_train.shape

In [ ]:
print(f"""Train NaN: 
{data_train.isnull().mean() * 100}

Test NaN: 
{data_test.isnull().mean() * 100}""")

In [ ]:
data_train.info()

In [ ]:
data_train["Product_Category_2"] = data_train["Product_Category_2"].fillna(data_train["Product_Category_2"].median()).astype("object")
data_train.drop("Product_Category_3", axis=1, inplace=True)
data_test["Product_Category_2"] = data_test["Product_Category_2"].fillna(data_test["Product_Category_2"].median()).astype("object")
data_test.drop("Product_Category_3", axis=1, inplace=True)
data_train.head()

In [ ]:
y = data_train.pop("Purchase")

In [ ]:
cat_cols = list(data_train.select_dtypes(include="object"))
oe = OrdinalEncoder()
for col in cat_cols:
    data_train[col] = oe.fit_transform(data_train[col].values.reshape(-1, 1))
    data_test[col] = oe.transform(data_test[col].values.reshape(-1, 1))
data_train.head()

In [ ]:
def comb_prod_cat(df): # Combine product category
    prod_cats = ["Product_Category_1", "Product_Category_2"]
    df["Total_Category"] = df[prod_cats[0]] + df[prod_cats[1]]
    return df.drop(prod_cats, axis=1)

In [ ]:
data_train = comb_prod_cat(data_train)
data_test = comb_prod_cat(data_test)
data_train.head()

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(27, 9.5))
r_idx = 0
c_idx = 0
for col in data_train:
    axes[r_idx][c_idx].set_title(f"{col} Distribution")
    data_train[col].hist(ax=axes[r_idx][c_idx])
    c_idx += 1
    if c_idx >= 4:
        r_idx += 1
        c_idx = 0
axes[r_idx][c_idx].set_title("Purchase (Target) Distribution")
axes[1][3] = y.hist(ax=axes[1][3])

In [ ]:
umap_model = umap.UMAP(n_components=2, n_neighbors=100, min_dist=0.01, n_jobs=-1, random_state=0)
sample_umap = pd.DataFrame(data=MinMaxScaler().fit_transform(data_train.sample(frac=0.1)), columns=data_train.columns)
data_umap = umap_model.fit_transform(sample_umap)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 8))
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_title("UMAP Dimensionality Reduced data_train; the redder the higher target value")
ax.scatter(data_umap[:,0], data_umap[:,1], c=y.loc[sample.index], marker=".", linewidths=1.75, cmap="YlOrRd")

In [ ]:
sample_vif = data_train.sample(frac=0.1)
vif_factors = [variance_inflation_factor(sample_vif.values, i) for i in range(sample_vif.shape[1])]

In [ ]:
fig, ax = plt.subplots(figsize=(29, 8))
ax.set_title("Variance Inflation Factor (Multicollinearity)")
pd.DataFrame({"Feature" : data_train.columns, "VIF" : vif_factors}).sort_values(by="VIF", ascending=False).plot.bar(x="Feature", y="VIF", ax=ax)

In [ ]:
out_model = COPOD(contamination=0.01, n_jobs=-1)
out_model.fit(data_train)
out_indices = pd.Series(np.argwhere(out_model.labels_ == 1).flatten())

In [ ]:
fig, ax = plt.subplots(figsize=(9, 8))
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_title("UMAP Dimensionality Reduced data_train; blue indicates inlier, orange indicates top 1 percent outliers")
ax.scatter(data_umap[:,0], data_umap[:,1], marker=".", linewidths=1)
data_x = pd.Series(data_umap[:,0])
data_y = pd.Series(data_umap[:,1])
ax.scatter(data_x[data_x.index.isin(out_indices)], data_y[data_y.index.isin(out_indices)], marker=".", linewidths=1, cmap="YlOrRd")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
sns.histplot(pd.DataFrame({"Outlier Score" : out_model.decision_scores_, "Outlier" : out_model.labels_}), x="Outlier Score", hue="Outlier", bins=225, multiple="stack", ax=ax)

In [ ]:
drop_idx = data_train.index[out_indices]
data_train = data_train.drop(out_indices, axis=0).reset_index(drop=True) # Remove top outliers
y = y.drop(drop_idx, axis=0).astype("float64")

In [ ]:
data_train["ID"] = pd.Series(np.arange(0, data_train.shape[0]))
data_test["ID"] = pd.Series(np.arange(0, data_test.shape[0]))
es_train = ft.EntitySet(id="sales")
es_train = es_train.entity_from_dataframe(entity_id="sales", dataframe=data_train, index="ID")
es_test = ft.EntitySet(id="sales")
es_test = es_test.entity_from_dataframe(entity_id="sales", dataframe=data_test, index="ID")

In [ ]:
primitives = ft.list_primitives()
agg_num_arr = ((primitives["type"] == "aggregation") & (primitives["valid_inputs"] == "Numeric"))
trans_num_arr = ((primitives["type"] == "transform") & (primitives["valid_inputs"] == "Numeric"))
num_agg_prim = list(primitives[agg_num_arr]["name"]) # Aggregation primitives whose valid input is numeric
num_trans_prim = list(primitives[trans_num_arr]["name"]) # Transform primitives whose valid input is numeric

In [ ]:
def generate_features(es):
    features, feature_names = ft.dfs(entityset=es, target_entity="sales", agg_primitives=num_agg_prim, trans_primitives=num_trans_prim, max_depth=1, max_features=500)
    features.replace([np.inf, -np.inf], np.nan, inplace=True) 
    return features, feature_names

In [ ]:
features_train, _ = generate_features(es_train)
features_test, _ = generate_features(es_test)

In [ ]:
remove_nan_threshold = 0.3
features_train = features_train.loc[:, features_train.isnull().mean() < remove_nan_threshold]
features_test = features_test.loc[:, features_test.isnull().mean() < remove_nan_threshold]

In [ ]:
ii = IterativeImputer(estimator=ElasticNet(normalize=True, random_state=0))
features_train = pd.DataFrame(data=ii.fit_transform(features_train), columns=features_train.columns)
features_test = pd.DataFrame(data=ii.transform(features_test), columns=features_test.columns)

In [ ]:
rs = RobustScaler()
features_train_scaled = pd.DataFrame(data=rs.fit_transform(features_train), columns=features_train.columns)
features_test_scaled = pd.DataFrame(data=rs.transform(features_test), columns=features_test.columns)

In [ ]:
def clean_col(name):
    return (
        str(name).strip().lower().replace("<", "").replace(">", "").replace(":", "").replace("feature", "")
    )

In [ ]:
features_train_scaled = features_train_scaled.rename(columns=clean_col)
features_test_scaled = features_test_scaled.rename(columns=clean_col)

In [ ]:
lgbm_baseline_params = { # LGBM regressor baseline (default) estimator parameters
    "num_iterations" : 100,
    "learning_rate" : 0.05,
    "max_depth" : 6,
    "num_leaves" : 255,
    "random_state" : 0
}
lgbm_feature_importance = lgb.LGBMRegressor(**lgbm_baseline_params)
lgbm_feature_importance.fit(features_train_scaled, y)

In [ ]:
fig, ax = plt.subplots(figsize=(27, 45))
lgb.plot_importance(lgbm_feature_importance, ax=ax)

In [ ]:
ee_feature_importance = ElasticNet(random_state=0)
ee_feature_importance.fit(features_train_scaled, y)

In [ ]:
fig, ax = plt.subplots(figsize=(27, 45))
ax.set_title("Feature importance")
list_coef, list_cols = [list(tuple) for tuple in zip(*sorted(zip(np.abs(ee_feature_importance.coef_), features_train_scaled.columns), reverse=False))]
pd.Series(data=list_coef, index=list_cols).plot.barh(ax=ax)

In [ ]:
feat_imp_combined = (np.abs((lgbm_feature_importance.feature_importances_ + ee_feature_importance.coef_)) / 2).flatten()
fig, ax = plt.subplots(figsize=(27, 45))
ax.set_title("Feature importance (LGBM feature importance and ElasticNet averaged)")
list_coef, list_cols = [list(tuple) for tuple in zip(*sorted(zip(feat_imp_combined, features_train_scaled.columns), reverse=False))]
pd.Series(data=list_coef, index=list_cols).plot.barh(ax=ax)

In [ ]:
lgbm_baseline_scores = cross_val_score(estimator=lgbm_baseline_estimator, X=features_train_scaled, y=y, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)

In [ ]:
print(f"""LGBM cross-validated RMSE baseline score
{np.mean(lgbm_baseline_scores)}""")

In [ ]:
lgbm_baseline_estimator.fit(features_train_scaled, y)

In [ ]:
perm_result = permutation_importance(estimator=lgbm_baseline_estimator, X=features_train_scaled, y=y, n_repeats=3, scoring="neg_root_mean_squared_error", n_jobs=3, random_state=0)

In [ ]:
fig, ax = plt.subplots(figsize=(27, 45))
ax.set_title("Permutation importance")
list_coef, list_cols = [list(tuple) for tuple in zip(*sorted(zip(perm_result.importances_mean, features_train_scaled.columns), reverse=False))]
pd.Series(data=list_coef, index=list_cols).plot.barh(ax=ax)

In [ ]:
selected_columns = [y for x, y in zip(list_coef, list_cols) if x > 0] # Drop columns which did not contribute in permutation score

In [ ]:
X_train = features_train_scaled[selected_columns] # Final data which is cleaned, feature engineered, scaled, feature selected, etc.
X_test = features_test_scaled[selected_columns]

In [ ]:
lgbm_reg = lgb.LGBMRegressor(**lgbm_baseline_params) # Final estimator object

In [ ]:
lgbm_after_feat_select = cross_val_score(estimator=lgbm_reg, X=X_train, y=y, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)

In [ ]:
print(f"""LGBM cross-validated RMSE after feature selection score
{np.mean(lgbm_after_feat_select)}""")

In [ ]:
params = {
    "num_iterations" : [100, 200, 300],
    "learning_rate" : [0.01, 0.02, 0.03],
    "max_depth" : [6, 9, 12],
    "num_leaves" : [63, 127, 255],
    "random_state" : [0]
}
gs_cv = GridSearchCV(estimator=lgbm_reg, param_grid=params, cv=3, scoring="neg_root_mean_squared_error", n_jobs=-1)
gs_cv.fit(X_train, y)

In [ ]:
print(f"""GridSearchCV results
Best Params: {gs_cv.best_params_}
Best Score: {gs_cv.best_score_}""")

In [ ]:
lgbm_reg.set_params(**gs_cv.best_params_)
lgbm_reg.fit(X_train, y)

In [ ]:
data_submission["Purchase"] = lgbm_reg.predict(X_test)

In [ ]:
data_submission.to_csv("results.csv", index=False, header=True)